In [16]:
import gym_examples
import gym
from stable_baselines3 import PPO
from sb3_contrib import MaskablePPO
from sb3_contrib.common.wrappers import ActionMasker
from sb3_contrib.common.maskable.policies import MaskableActorCriticPolicy
from gym.wrappers import FlattenObservation

In [10]:
gym.__version__

'0.26.2'

In [11]:
env = gym.make("gym_examples/StarWars-v0")

In [12]:
start_state = env.reset()
start_state



({'r2d2': array([0, 2]), 'c3po': array([4, 0]), 'vader': array([0, 4])},
 {'c3po_distance': 6.0, 'vader_distance': 2.0})

In [13]:
env.action_space

Discrete(4)

In [14]:
def masking_fn(env: gym.Env):
    return env.get_valid_action_mask()

In [17]:
env = FlattenObservation(ActionMasker(env, masking_fn))

In [18]:
# env = ActionMasker(env, masking_fn)
model = MaskablePPO(MaskableActorCriticPolicy, env, verbose=1)
model.learn(total_timesteps=10)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 22.7     |
|    ep_rew_mean     | -0.0401  |
| time/              |          |
|    fps             | 584      |
|    iterations      | 1        |
|    time_elapsed    | 3        |
|    total_timesteps | 2048     |
---------------------------------
